# Sparse fingerprints with rdkit and scipy.sparse



In [1]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

import tqdm

import numpy as np
from scipy import sparse
import pandas as pd

import time


#autocomplete wasn't working for some reason. This fixes it. 
%config Completer.use_jedi = False

In [2]:
# random 10k smiles from the enamine lead-like diversity set
# dont really pandas but ok.
smi_df = pd.read_csv('enamine_leadlike_10k.smi', sep='\t', header=None)
smi_df.head()

,0
0,CC[C@@H]1CN[C@H](C(=O)N2CC(N)CC3(CCC3)C2)C1
1,CC1=C(C(=O)N2CC([C@H](C)NC(=O)C3(CO)CCCC3)C2)C...
2,CC1CCC2(CC2)N(C(=O)C2=CC=C(OC3CC3)C=C2)C1
3,CC(=O)CCCC(=O)NC1CN(C(=O)C2(C3CCC3)CC2)CC12CCC2
4,CCCC(C)(C)C(=O)N[C@H]1[C@@H]2CN(C(=O)C(CC(C)C)...


In [3]:
def make_fingerprints(smiles):
    """Parse list of smiles into fingerprints, stored as sparse matrix.
    Currently using size 65536 - this is usually way too large, 
    but it leaves room to move. Plus, there's no loss in 
    memory from using large sizes due to the sparsity. 
    
    There is a folding function to get back to common usage sizes. 
    """

    #see FPSim2 for these parameters
    fingerprint_function = rdMolDescriptors.GetMorganFingerprintAsBitVect
    pars = { "radius": 2,
                     "nBits": 65536,
                     "invariants": [],
                     "fromAtoms": [],
                     "useChirality": False,
                     "useBondTypes": True,
                     "useFeatures": False,
            }

    #store bit indices in these:
    row_idx = list()
    col_idx = list()
    
    count = 0
    
    #iterate through smiles, 
    for smi in tqdm.tqdm_notebook(smiles, smoothing=0):
        try:
            mol = Chem.MolFromSmiles(smi)
            fp = fingerprint_function(mol, **pars)
        
            #if the smiles failed to parse, it would have given an exception by now.
            onbits = list(fp.GetOnBits())
            #these bits all have the same row:
            row_idx += [count]*len(onbits)
            #and the column indices of those bits:
            col_idx+=onbits
            #update the count
            count+=1
        except KeyboardInterrupt:
            raise
        except:
            print('smiles failed')

    
    #generate a sparse matrix out of the row,col indices:
    unfolded_size = 65536
    
    fingerprint_matrix = sparse.coo_matrix((np.ones(len(row_idx)).astype(bool), (row_idx, col_idx)), 
                      shape=(max(row_idx)+1, unfolded_size))
    
    #convert to csr matrix, smaller and usually faster too:
    fingerprint_matrix =  sparse.csr_matrix(fingerprint_matrix)

    return fingerprint_matrix

In [4]:
fp_mat = make_fingerprints(smi_df[0])

/Users/ljmartin/miniconda3/envs/p4env/lib/python3.7/site-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
##If you wish to fold it down to a better size for sklearn or something, use this.

def fold_fingerprints(feature_matrix):
    """Folds a fingerprint matrix by bitwise OR.
    (scipy will perform the bitwise OR because the `data` is bool,
    and it will not cast it to int when two Trues are added."""

    ncols = feature_matrix.shape[1]
    return feature_matrix[:,:ncols//2] + feature_matrix[:,ncols//2:]

def fold_to_size(feature_matrix, size):
    """Performs the `fold` operation multiple times to reduce fp 
    length to the desired size."""

    while feature_matrix.shape[1]>size:
        feature_matrix = fold_fingerprints(feature_matrix)

    return feature_matrix

In [6]:
# for example:
fold_to_size(fp_mat, 2<<10) #outputs a matrix with dimensionality 2048

<10000x2048 sparse matrix of type '<class 'numpy.bool_'>'
	with 444039 stored elements in Compressed Sparse Row format>

# Verify the sparse matrix is the right shape, and is really just made up of three numpy arrays

In [7]:
#This
print('Fingerprint matrix shape:', fp_mat.shape)
print('\n')
print('Indices:', fp_mat.indices)
print('Indices shape:', fp_mat.indices.shape)
print('\n')
print('Index pointer:', fp_mat.indptr)
print('Index pointer shape:', fp_mat.indptr.shape)
print('\n')
print('Actual data (these are all just "ON" bits!):', fp_mat.data)
print('Actual data shape:', fp_mat.data.shape)

Fingerprint matrix shape: (10000, 65536)


Indices: [ 1701  5258  6647 ... 64138 64505 65272]
Indices shape: (451259,)


Index pointer: [     0     43     89 ... 451165 451212 451259]
Index pointer shape: (10001,)


Actual data (these are all just "ON" bits!): [ True  True  True ...  True  True  True]
Actual data shape: (451259,)


# Calculate Jaccard or Dice metrics

Jaccard or Dice distances (as with Tanimoto) are simply [set operations](https://en.wikipedia.org/wiki/Set_(mathematics)#Basic_operations) - i.e. intersections, unions, cardinalities.

Sparse matrices have a huge advantage calculating these properties. Since we know any absent bit is by definition not included in the set of bits for a molecule, we can ignore any calculations involving that absent bit. Some linear algebra calculations using sparse matrices are faster for exactly that reason - they ignore the zero values. 

For instance, the pairwise intersection matrix between two fingerprint matrices is simply the dot product. From this matrix, Dice or Jaccard can be calculated fairly straightforwardly.

I get about 5 seconds for a 10k by 10k Dice distance matrix. That's 100 million fingerprints, so **`50`** nanoseconds per similarity. 


As a rough comparison, chemfp [gets](https://chemfp.com/performance/#k10_performance) 1 million Tanimoto similarities done in 16.0 milliseconds for size 2,048. That's an impressive **`16 ns`** per similarity. But remember chemfp similarities scale linearly in fingerprint size, and the sparse approach is using size 65528, in which case chemfp might be expected to perform 32X slower, or **`512`** ns. 

In [8]:
def fast_dice(X, Y=None):
    if isinstance(X, np.ndarray):
        X = sparse.csr_matrix(X).astype(bool).astype(int)
    if Y is None:
        Y = X
    else:
        if isinstance(Y, np.ndarray):
            Y = sparse.csr_matrix(Y).astype(bool).astype(int)
            
    intersect = X.dot(Y.T)
    #cardinality = X.sum(1).A
    cardinality_X = X.getnnz(1)[:,None] #slightly faster on large matrices - 13s vs 16s for 12k x 12k
    cardinality_Y = Y.getnnz(1) #slightly faster on large matrices - 13s vs 16s for 12k x 12k
    return (1-(2*intersect) / (cardinality_X+cardinality_Y.T)).A


def fast_jaccard(X, Y=None):
    """credit: https://stackoverflow.com/questions/32805916/compute-jaccard-distances-on-sparse-matrix"""
    if isinstance(X, np.ndarray):
        X = sparse.csr_matrix(X)
    if Y is None:
        Y = X
    else:
        if isinstance(Y, np.ndarray):
            Y = sparse.csr_matrix(Y)
    assert X.shape[1] == Y.shape[1]

    X = X.astype(bool).astype(int)
    Y = Y.astype(bool).astype(int)
    intersect = X.dot(Y.T)
    x_sum = X.sum(axis=1).A1
    y_sum = Y.sum(axis=1).A1
    xx, yy = np.meshgrid(x_sum, y_sum)
    union = ((xx + yy).T - intersect)
    return (1 - intersect / union).A

In [9]:



start = time.time()
fast_dice(fp_mat)
end = time.time()

tot_time = end-start
print('Total time in seconds:', end-start)

Total time in seconds: 5.027843952178955


In [10]:
print('Nanoseconds per similarity:', (tot_time / (10_000**2)) / 1e-9)

Nanoseconds per similarity: 50.27843952178955


# Memory usage

Sparse matrices are small - that's their whole thing. There's a way to make this smaller, but for now I estimate most people could hold about 10 million to 100 million smiles in RAM.

In [11]:
def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n]+'bytes'

In [12]:
b = fp_mat.indptr.nbytes + fp_mat.data.nbytes + fp_mat.indices.nbytes
print('10,000 smiles:', format_bytes(b))
print('1 million smiles:', format_bytes(b*1000))


10,000 smiles: (2.1899213790893555, 'megabytes')
1 million smiles: (2.1385950967669487, 'gigabytes')
